# ПРИКЛАДИ
## Метод головних компонентів (Principal Component Analysis, PCA)


Правильний порядок дій у методі головних компонент:
* Нормування
* Обчислення коварійної матриці
* Знаходження власних чисел та власних значень
* Визначення власних векторів із максимальним вкладом (за величиною власних чисел)
* Збільшення вихідної матриці на матрицю власних векторів

# Приклад
Почнемо ми зі спрощеної ситуації. Припустимо, що у нас є одна реальна ознака, він буде рівномірно розподілений.

Ми додамо 2 ознака, значення якого дорівнює константі.

У реальній ситуації очевидно, що це сміття, однак, якщо ознак багато, не завжди вдається це помітити, так само відсікання такої ознаки може бути утруднено, якщо це не 100% константа (наприклад, частина значень все ж таки відрізняються).

In [13]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go

x = np.arange(0, 10, 0.1)
Data = pd.DataFrame()
Data['feature1'] = x
Data['feature2'] = [x.mean()]*len(x) # constant

fig = go.Figure()
fig.add_trace(go.Scatter(x=Data['feature1'], y=Data['feature2'], mode='markers'))
fig.show()

In [14]:
display(Data.head())

,feature1,feature2
0,0.0,4.95
1,0.1,4.95
2,0.2,4.95
3,0.3,4.95
4,0.4,4.95


**Перше, що необхідно зробити, це стандартизувати (нормалізувати) дані**.

Метод дуже чутливий до ненормованих даних, тому необхідно використати формулу:

$$x_{std} = \frac{x - E}{\sigma}$$
    
де, $𝐸$ - математичне очікування (для експериментальних даних його апроксимація у вигляді середнього)

$𝜎$ - стандартне (середньоквадратичне) відхилення

Напишемо для цього функцію, але врахуємо невелику особливість. Якщо дані константні, то відхилення дорівнює 0, тож нічого хорошого не вийде. І тут примусово нормуємо значення до 0.

Врахуємо, що такі величини в Python обчислюються чисельними методами і можуть бути відмінними від 0 (але фактично $10^{-15}$ ступеня це 0 і є)

In [11]:
def normolize(col):
    if abs(col.std(ddof=0)) < 10**(-10):
        return 0
    return (col - col.mean()) / col.std(ddof=0)
Data = Data.apply(normolize)
Data.head()

,feature1,feature2
0,-1.714816,0
1,-1.680173,0
2,-1.645531,0
3,-1.610888,0
4,-1.576245,0


Аналог цієї операції - [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

```Python
import StandardScaler
Data = StandardScaler().fit_transform(Data)

# import sklearn
# Data = sklearn.preprocessing.StandardScaler().fit_transform(Data)
```
Тепер пригадаємо, що спектральне розкладання – це розкладання квадратної матриці. А наша матриця ознак прямокутна (100 на 2)

Це легко виправити, адже для PCA знадобиться не матриця вихідних ознак (нехай і нормованих), а матриця коварації, яка обчислюється за формулою:

$\frac{1}{n-1}\cdot \left ( \left ( X - \tilde{x}\right )^{T} \cdot \left ( X - \tilde{x}\right ) \right )$


In [27]:
np.dot(np.transpose(Data - Data.mean().mean()), Data - Data.mean().mean()) / (len(Data)-1)
# Навіщо двічі застосовувати метод .mean()?:
# Перший раз застосовуємо середнє - отримуємо колонку із середніх для кожної ознаки.
# Другий раз усереднюємо середні.

array([[1.01010101e+00, 1.55630703e-34],
       [1.55630703e-34, 1.79286569e-34]])

У Numpy є спеціальна функція для розрахунку [матриці коваріації](https://nbviewer.org/github/pruhlo/notes-for-Data-Science/blob/master/Correlation.ipynb#part1):

In [28]:
np.cov(np.transpose(Data))

array([[1.01010101, 0.        ],
       [0.        , 0.        ]])

Розрахована вручну коваріаційна матриця по суті така сама, як і отримана за допомогою https://numpy.org/doc/stable/reference/generated/numpy.cov.html ($10^{-34}$ це насправді 0).

Для краси візьмемо "кругліші" значення.

In [30]:
cov_matrix = np.cov(np.transpose(Data))

Використаємо [numpy.linalg.eigh](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eigh.html) як у [відео](https://stepik.org/lesson/272591/step/2) для отримання власних значень та власних векторів

In [32]:
eigh_val, eigh_vec = np.linalg.eigh(cov_matrix)

print('Власні числа: ', eigh_val, sep='\n')
print('Власні вектори: ', eigh_vec, sep='\n')

Власні числа: 
[0.         1.01010101]
Власні вектори: 
[[0. 1.]
 [1. 0.]]


Власні числа показують який "розкид" кожної з компонент, отже яка частка корисної інформації зберігається у ній.

Вочевидь, що у разі вся корисна інформація зберігалася у одному стовпці, т.к. другий був заповнений константою.

Вклад кожного з оригінальних ознак тим більше, що більше власне число, відповідне, тому розрахуємо величину вкладу:

In [33]:
print('Вклад кожної компоненти в "розкид" ознак (у %): ', [i/sum(eigh_val)*100 for i in eigh_val])

Вклад кожної компоненти в "розкид" ознак (у %):  [0.0, 100.0]


Виберемо лише ті власні вектори, які відповідають найбільшому вкладу (тобто найбільшим власним числам).

У цьому прикладі це буде 1 вектор:

In [35]:
eigh_vec[:, 1]

array([1., 0.])

І нарешті отримаємо з датасета "очищені" дані, помноживши вихідну матрицю (з нормованими даними) на власний вектор, що цікавить нас:

In [37]:
# Data.dot(eigh_vec[:, 1:])

це просто вихідний нормований стовпець, перш ніж ми додали стовпець з константою.

# Приклад 2
Ускладнимо.
Додамо колонку із випадковими значеннями ознаки.

In [39]:
Data = pd.DataFrame()
Data['feature1'] = x
Data['feature2'] = [x.mean()]*len(x)
Data['feature3'] = np.random.uniform(size=100)

Data.head()

,feature1,feature2,feature3
0,0.0,4.95,0.254878
1,0.1,4.95,0.931229
2,0.2,4.95,0.679504
3,0.3,4.95,0.863679
4,0.4,4.95,0.166317


Побудуємо візуалізацію розподілу ознак у 3D:

In [40]:
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=Data['feature1'], y=Data['feature2'], z=Data['feature3'], mode='markers'))
fig.show()

Можливо в цьому випадку не дуже інформативна картинка. Крім того, якщо ознак більше 3, то їхній взаємозв'язок вже не покажеш візуально (можна використовувати розмір маркера або колір, але його складно порівнювати з геометричним положенням, та й підходять колір і розмір більше для "відповіді").

Тому існує інший вид діаграм:

Pairplot у seaborn - https://seaborn.pydata.org/generated/seaborn.pairplot.html

Scatterplot в plotly - https://plotly.com/python/splom/

По суті, це матриця з попарних плоских графіків розподілів ознак.

Слід зазначити, що з діагоналі цієї матриці завжди будуть марні графіки розподілу ознаки щодо себе.

Ми скористаємося Plotly:

In [41]:
fig = go.Figure()

fig.add_trace(go.Splom(dimensions=[
    dict(label='feature1', values=Data['feature1']),
    dict(label='feature2', values=Data['feature2']),
    dict(label='feature3', values=Data['feature3'])
],))

fig.show()

In [42]:
#Нормуємо дані:
Data = Data.apply(normolize)
Data.head()

,feature1,feature2,feature3
0,-1.714816,0,-0.863149
1,-1.680173,0,1.506395
2,-1.645531,0,0.624496
3,-1.610888,0,1.269739
4,-1.576245,0,-1.173417


In [43]:
cov_matrix = np.cov(np.transpose(Data))

eigh_val, eigh_vec = np.linalg.eigh(cov_matrix)
print('Власні числа: ', eigh_val, '\n', sep='\n')
print('Власні вектори: ', eigh_vec, sep='\n')

print('Вклад кожної компоненти в "розкид" ознак (у %): ', [i/sum(eigh_val)*100 for i in eigh_val])

Власні числа: 
[0.         0.98624358 1.03395844]


Власні вектори: 
[[ 0.         -0.70710678 -0.70710678]
 [ 1.          0.          0.        ]
 [ 0.         -0.70710678  0.70710678]]
Вклад кожної компоненти в "розкид" ознак (у %):  [0.0, 48.819057305905275, 51.180942694094725]


Неприємне відкриття, чи не так? Якщо ваші дані зашумлені, то **Метод Головних Компонент** може визначити саме шум як значну частину інформації. Він ніяк не враховує "відповідь", а аналізує лише "ознаки", тому ознака з гарним розкидом (наприклад, рівномірний шум) буде прийнята за корисну інформацію.

# Приклад 3
Задамо 2 і 3 ознаки через 1, після чого забудемо про це.
У разі це буде проста лінійна залежність, тобто. насправді якась пряма в 3D.

In [44]:
Data = pd.DataFrame()
Data['feature1'] = x
Data['feature2'] = 2*x-1
Data['feature3'] = -0.5*x + 10
display(Data.head())

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=Data['feature1'], y=Data['feature2'], z=Data['feature3'], mode='markers'))
fig.show()

fig = go.Figure()
fig.add_trace(go.Splom(dimensions=[
    dict(label='feature1', values=Data['feature1']),
    dict(label='feature2', values=Data['feature2']),
    dict(label='feature3', values=Data['feature3'])
],))
fig.show()

,feature1,feature2,feature3
0,0.0,-1.0,10.00
1,0.1,-0.8,9.95
2,0.2,-0.6,9.90
3,0.3,-0.4,9.85
4,0.4,-0.2,9.80


In [45]:
Data = Data.apply(normolize)
cov_matrix = np.cov(np.transpose(Data))

eigh_val, eigh_vec = np.linalg.eigh(cov_matrix)
print('Власні числа: ', eigh_val, '\n', sep='\n')
print('Власні вектори: ', eigh_vec, sep='\n')

print('Вклад кожної компоненти в "розкид" ознак (у %): ', [i/sum(eigh_val)*100 for i in eigh_val])

Власні числа: 
[-1.68377698e-16  3.90422303e-16  3.03030303e+00]


Власні вектори: 
[[-0.68248558 -0.44819649  0.57735027]
 [-0.04690676  0.8151481   0.57735027]
 [-0.72939234  0.3669516  -0.57735027]]
Вклад кожної компоненти в "розкид" ознак (у %):  [-5.556464043096976e-15, 1.2883936005623003e-14, 100.0]


оу! Та це ж чудова новина! Наш метод зрозумів, що між ознаками є залежність і просто відсік 2 з 3!

До речі, якщо ми вирішимо, що все ж таки відкидати 2 ознаки занадто марнотратно і викинемо тільки 1, то після множення матриці на власні вектори, що відповідають компонентам з максимальним розкидом, ми побачимо, що значення "головної" компоненти значно більше, ніж значення "другорядної"

In [46]:
#Data.dot(eigh_vec[:, 1:])

# Приклад 4
Задамо 2 і 3 ознаки через 1, після чого так само забудемо про це, але цього разу залежність ознаки 3 від 1 буде не лінійною, а квадратичною.

Легко здогадатися, що дива не станеться, PCA не може повною мірою компенсувати таку залежність, так як матричні операції еквівалентні комбінації 2 простих:

Поворот осей на деякий кут

Перенесення початку координат до якоїсь точки

In [47]:
Data = pd.DataFrame()
Data['feature1'] = x
Data['feature2'] = 2*x-1
Data['feature3'] = x**2
display(Data)

fig = go.Figure()
fig.add_trace(go.Scatter3d(x=Data['feature1'], y=Data['feature2'], z=Data['feature3'], mode='markers'))
fig.show()

fig = go.Figure()
fig.add_trace(go.Splom(dimensions=[
    dict(label='feature1', values=Data['feature1']),
    dict(label='feature2', values=Data['feature2']),
    dict(label='feature3', values=Data['feature3'])
],))
fig.show()

,feature1,feature2,feature3
0,0.0,-1.0,0.00
1,0.1,-0.8,0.01
2,0.2,-0.6,0.04
3,0.3,-0.4,0.09
4,0.4,-0.2,0.16
...,...,...,...
95,9.5,18.0,90.25
96,9.6,18.2,92.16
97,9.7,18.4,94.09
98,9.8,18.6,96.04


In [49]:
Data = Data.apply(normolize)
cov_matrix = np.cov(np.transpose(Data))

eigh_val, eigh_vec = np.linalg.eigh(cov_matrix)
print('Власні числа: ', eigh_val, '\n', sep='\n')
print('Власні вектори: ', eigh_vec, sep='\n')

print('Вклад кожної компоненти в "розкид" ознак (у %): ', [i/sum(eigh_val)*100 for i in eigh_val])

# Data.dot(eigh_vec[:, 1:])

Власні числа: 
[-1.99176677e-16  4.34959278e-02  2.98680710e+00]


Власні вектори: 
[[-7.07106781e-01 -4.05220539e-01 -5.79479348e-01]
 [ 7.07106781e-01 -4.05220539e-01 -5.79479348e-01]
 [ 3.60822483e-15  8.19507553e-01 -5.73068382e-01]]
Вклад кожної компоненти в "розкид" ознак (у %):  [-6.572830356259882e-15, 1.4353656167786872, 98.56463438322133]


Найбільшого внеску дала "квадратична" компонента, т.к. її розкид найбільший навіть після нормування, але й невеликий шматочок "лінійної" компоненти залишився.

# Автоматизація процесу за допомогою Scikit-learn
у Python є бібліотека, яка відповідає за деякі методи машинного навчання, в якій є і метод головних компонентів.

Перехід до нової матриці з виділеними компонентами виглядає так:

In [50]:
from sklearn import decomposition
pca = decomposition.PCA()
pca.fit(Data)
pca.transform(Data)

array([[-2.62454320e+00,  4.78621983e-01, -2.22044605e-16],
       [-2.58419964e+00,  4.50823566e-01, -2.22044605e-16],
       [-2.54346800e+00,  4.23580127e-01, -2.22044605e-16],
       [-2.50234827e+00,  3.96891666e-01, -2.22044605e-16],
       [-2.46084045e+00,  3.70758183e-01, -2.22044605e-16],
       [-2.41894454e+00,  3.45179679e-01, -2.22044605e-16],
       [-2.37666055e+00,  3.20156152e-01,  0.00000000e+00],
       [-2.33398846e+00,  2.95687604e-01, -2.22044605e-16],
       [-2.29092830e+00,  2.71774034e-01, -2.22044605e-16],
       [-2.24748004e+00,  2.48415442e-01, -1.11022302e-16],
       [-2.20364370e+00,  2.25611828e-01, -1.11022302e-16],
       [-2.15941927e+00,  2.03363192e-01, -1.11022302e-16],
       [-2.11480675e+00,  1.81669535e-01, -2.22044605e-16],
       [-2.06980614e+00,  1.60530855e-01, -1.11022302e-16],
       [-2.02441745e+00,  1.39947154e-01, -2.22044605e-16],
       [-1.97864067e+00,  1.19918431e-01, -1.11022302e-16],
       [-1.93247580e+00,  1.00444686e-01

In [51]:
# При цьому можна подивитися як виглядає внесок кожної з компонентів у розкид вихідних даних:

pca.explained_variance_ratio_

array([9.85646344e-01, 1.43536562e-02, 4.80801145e-33])

Після чого можна ухвалити рішення, що нам потрібні 2 компоненти, які забезпечують майже 100% даних про ознаки (98,6% + 1,4%).

Для цього при створенні моделі необхідно просто вказати скільки компонентів ми хочемо отримати в результаті: n_components = 2

In [54]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=2)
pca.fit(Data)
pca.transform(Data)
pca.explained_variance_ratio_

array([0.98564634, 0.01435366])

Легко переконатися, що значення в матриці ознак, так і відсотки від вкладу в розкид вийшли практично ідентичні як в "ручному", так і в "автоматизованому" рішенні.

# Приклад 5.
Створимо 2 ознаки, що так само мають зв'язок, проте не лінійну залежність.

Розглянемо наш лінійний ряд чисел від 0 до 10 із кроком 0.1. Згенеруємо парами значення ознак так, щоб сума значень у кожній парі давала одне із чисел нашого ряду. Майже як банківський рахунок, якщо ЗП виплачують 2 рази на місяць і випадкова частина приходить у 1 половині, а залишок у 2.

In [56]:
Data = pd.DataFrame()
Data['feature1'] = [np.random.uniform(high=i) for i in x]
Data['feature2'] = x - Data['feature1']
print("Ознаки: ")
display(Data)
print("Сума значень ознак: ")
display(Data['feature1'] + Data['feature2'])

fig = go.Figure()
fig.add_trace(go.Scatter(x=Data['feature1'], y=Data['feature2'], mode='markers'))
fig.update_layout(
    width=500,
    height=500,
)
fig.show()

Ознаки: 


,feature1,feature2
0,0.000000,0.000000
1,0.062660,0.037340
2,0.164107,0.035893
3,0.258486,0.041514
4,0.275217,0.124783
...,...,...
95,1.714680,7.785320
96,4.383698,5.216302
97,7.552136,2.147864
98,4.097070,5.702930


Сума значень ознак: 


0     0.0
1     0.1
2     0.2
3     0.3
4     0.4
     ... 
95    9.5
96    9.6
97    9.7
98    9.8
99    9.9
Length: 100, dtype: float64

Очевидно, що лінійного зв'язку між ознаками немає. Однак так само очевидно, що елементарне перетворення дозволило б позбавитися 2 вихідних ознак і перейти до 1 "синтетичного".

In [57]:
Data = Data.apply(normolize)
cov_matrix = np.cov(np.transpose(Data))

eigh_val, eigh_vec = np.linalg.eigh(cov_matrix)
print('Власні числа: ', eigh_val, '\n', sep='\n')
print('Власні вектори: ', eigh_vec, sep='\n')

print('Вклад кожної компоненти в "розкид" ознак (у %): ', [i/sum(eigh_val)*100 for i in eigh_val])

Власні числа: 
[0.88835914 1.13184288]


Власні вектори: 
[[-0.70710678 -0.70710678]
 [-0.70710678  0.70710678]]
Вклад кожної компоненти в "розкид" ознак (у %):  [43.97377734358605, 56.02622265641396]


Однак з точки зору PCA обидві ознаки важливі та містять корисну інформацію. Метод не може зробити диво, він не знає про внутрішній зв'язок між ознаками.

Повторимося, по суті він повертає і зсуває осі, а за вихідним розподілом ознак не можна сказати, що такий поворот призведе до чогось корисного.

Тому іноді зменшити розмірність вихідних даних можна знаючи про їхню структуру, проте метод головних компонент цієї структури не побачить.

# Іриси Фішера
[Wiki](https://uk.wikipedia.org/wiki/Іриси_Фішера)

Скористаємося класичним прикладом із Ірисами Фішера. Сам датасет доступний за адресою https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data

In [58]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', 
                 header=None,
                 names=["sepal_length", "sepal_width", "petal_length", "petal_width", "species"])
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Візуалізуємо залежність ознак щодо один одного:

In [60]:
import plotly.express as px
fig = px.scatter_matrix(df, 
                        dimensions=["sepal_width", "sepal_length", "petal_width", "petal_length"],
                        color="species")
fig.show()

In [62]:
Data = df[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
Data = Data.apply(normolize)
display(Data)
cov_matrix = np.cov(np.transpose(Data))
print('Коваріаційна матриця  :', cov_matrix, '\n', sep='\n')
eigh_val, eigh_vec = np.linalg.eigh(cov_matrix)


print('Власні числа: ', eigh_val, '\n', sep='\n')
print('Власні вектори: ', eigh_vec, '\n', sep='\n')

print('Вклад кожної компоненти в "розкид" ознак (%):\n', [i/sum(eigh_val)*100 for i in eigh_val])

,sepal_length,sepal_width,petal_length,petal_width
0,-0.900681,1.032057,-1.341272,-1.312977
1,-1.143017,-0.124958,-1.341272,-1.312977
2,-1.385353,0.337848,-1.398138,-1.312977
3,-1.506521,0.106445,-1.284407,-1.312977
4,-1.021849,1.263460,-1.341272,-1.312977
...,...,...,...,...
145,1.038005,-0.124958,0.819624,1.447956
146,0.553333,-1.281972,0.705893,0.922064
147,0.795669,-0.124958,0.819624,1.053537
148,0.432165,0.800654,0.933356,1.447956


Коваріаційна матриця  :
[[ 1.00671141 -0.11010327  0.87760486  0.82344326]
 [-0.11010327  1.00671141 -0.42333835 -0.358937  ]
 [ 0.87760486 -0.42333835  1.00671141  0.96921855]
 [ 0.82344326 -0.358937    0.96921855  1.00671141]]


Власні числа: 
[0.02074601 0.14834223 0.92740362 2.93035378]


Власні вектори: 
[[ 0.26199559  0.72101681  0.37231836 -0.52237162]
 [-0.12413481 -0.24203288  0.92555649  0.26335492]
 [-0.80115427 -0.14089226  0.02109478 -0.58125401]
 [ 0.52354627 -0.6338014   0.06541577 -0.56561105]]


Вклад кожної компоненти в "розкид" ознак (%):
 [0.5151926808906327, 3.6838319576273912, 23.03052326768062, 72.77045209380134]


Давайте тепер виділимо 3 найбільш значущі компоненти і побудуємо 3D графік залежності нових синтетичних ознак (нагадаємо, у процесі трансформації ми повернули і зрушили осі, тому некоректно говорити, що 3 нові ознаки будуть "sepal_width", "petal_length" та "petal_width")

In [63]:
Data_PCA = Data.dot(eigh_vec[:, 1:])
Data_PCA['species'] = df['species']
Data_PCA

fig = go.Figure()
for spec in Data_PCA['species'].unique():
    Data_PCA_spec = Data_PCA[Data_PCA['species']==spec]
    fig.add_trace(go.Scatter3d(x=Data_PCA_spec[0], y=Data_PCA_spec[1], z=Data_PCA_spec[2], mode='markers', name=spec))
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

Якщо у вихідних даних при використанні практично будь-якої пари ознак хоча б 1 квітка одного сорту виявлялася оточена "чужинцями", то тепер групи майже не перетинаються (це правда не означає, що можна легко провести кордон між сортами без виділення кольором). аналізу якого сорту конкретна квітка нам все одно довелося б ще споруджувати якийсь алгоритм, навчати його на нових даних, проте точність такого алгоритму майже напевно була б вищою, ніж під час використання сирих даних.

P.S. Порівняйте, до речі, наші результати з результатом https://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_iris.html

In [64]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=3)
pca.fit(Data)
pca.transform(Data)
pca.explained_variance_ratio_

array([0.72770452, 0.23030523, 0.03683832])